In [1]:
#import sys
#sys.path.insert(0, "/home/martial/Data/programs/Github/hyperspy_T/")
import hyperspy.api as hs

In [2]:
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import dask.array as da                # Import dask to wrap

In [3]:
%matplotlib qt4

In [4]:
data_file = hs.signals.Signal2D(np.empty((5,5,264,264),dtype=float)).as_lazy()

In [ ]:
data_file.save('Data_10.hdf5')

In [ ]:
data_file = hs.load("Data_10.hdf5", lazy=True)

In [ ]:
data_file.data #numpy array
#you have to put the raw data (*.mat) in this array

In [ ]:
data_file.data[i,j,k,l]= matlab_data[i2,j2,k2,l2]

In [5]:
data_file.plot()

### Load data

In [ ]:
specImg_hl = hs.load("Data_test/Dual_EELS_test_data/EELS Spectrum Image (high-loss).dm4")

In [ ]:
specImg_ll = hs.load("Data_test/Dual_EELS_test_data/EELS Spectrum Image (low-loss).dm4")

In [ ]:
# Live STEM image
specImg_abf_live = hs.load("Data_test/Dual_EELS_test_data/ADF Image.dm4")

In [ ]:
# Reshape live data to the size of the EELS datacube (it is not the case when using sub-pixel option)
specImg_abf_live_eels = specImg_abf_live.rebin((specImg_ll.data.shape[0],specImg_ll.data.shape[1]))

In [ ]:
# Survey image
survey_img = hs.load('Data_test/Dual_EELS_test_data/ADF Image (SI Survey).dm4')

# To clean the survay image, optional
#survey_img.data = sc.ndimage.filters.gaussian_filter(survey_img.data, sigma=3)
#survey_img.save('survey_img_Gauss3.png')

### Plot Data

In [ ]:
specImg_ll.plot(navigator=specImg_abf_live_eels)

In [ ]:
specImg_hl.plot(navigator=specImg_abf_live_eels)

### Alignment Low loss data

In [ ]:
specImg_ll.align_zero_loss_peak(also_align=[specImg_hl], subpixel=True)

In [ ]:
specImg_ll.plot(navigator=specImg_abf_live_eels)

In [ ]:
specImg_hl.plot(navigator=specImg_abf_live_eels)

### BG subtraction and vertical integration

In [ ]:
Ti_BG_sub = specImg_hl.remove_background((380,445), fast=False)

In [ ]:
Ti_BG_sub.plot()

In [ ]:
Ti_BG_sub_L3 = Ti_BG_sub.integrate_in_range((454.,460.))
Ti_BG_sub_L2 = Ti_BG_sub.integrate_in_range((460.,470.))

In [ ]:
Ti_BG_sub_L3.plot()

In [ ]:
Ti_BG_sub_L2.plot()

In [ ]:
(Ti_BG_sub_L3/Ti_BG_sub_L2).plot()

In [ ]:
# Mean value of the L3 to L2 ratio (to be compared with other maps)
(Ti_BG_sub_L3/Ti_BG_sub_L2).data.mean()

### N map

In [ ]:
# Remove BG in front of the N edge
N_BG_sub = specImg_hl.remove_background((372.,400.), fast=False)

In [ ]:
N_BG_sub.plot()

In [ ]:
N_BG_sub_map =N_BG_sub.integrate_in_range((400.,420.))

In [ ]:
N_BG_sub_map.plot()

In [ ]:
N_BG_average = specImg_hl.integrate1D(0).integrate1D(0)

In [ ]:
N_BG_average.plot()

In [ ]:
N_BG_average_BG_sub =N_BG_average.remove_background((372.,400.), fast=False)

In [ ]:
N_BG_average_BG_sub.plot()

In [ ]:
N_BG_average_BG_sub.crop_signal1D((372.,440.))

In [ ]:
N_BG_average_BG_sub.plot()

# N peak is clearly visible:-)

In [ ]:
# Following not implemented for this example

### Denoising by PCA

In [ ]:
specImg_hl.decomposition(normalize_poissonian_noise=True, algorithm='svd')

In [ ]:
specImg_hl.plot_explained_variance_ratio()

In [ ]:
specImg_hl_PCA8 = specImg_hl.get_decomposition_model(components=8)

In [ ]:
specImg_hl_PCA8.plot(navigator=specImg_abf_live)

### Deconvolution after PCA

In [ ]:
specImg_hl_PCA8_deconv = specImg_hl_PCA8.richardson_lucy_deconvolution(psf=specImg_ll, iterations=5)

In [ ]:
specImg_hl_PCA8_deconv.plot(navigator=specImg_abf_live)

### BG sub and Si/C integration on integrated data

#### integration along x axis

In [ ]:
specImg_hl_temp1 = specImg_hl.deepcopy()

In [ ]:
#rebin_non_constant_interval
ini_interval = (1, 6, 11, 16, 21, 26, 31, 40, 50, 60)
axis = 0

In [ ]:
fig, ax = plt.subplots()
y_size = specImg_abf_live.axes_manager[1].size
x=np.empty(y_size)
for i in range(0, np.size(ini_interval)):
    x.fill(ini_interval[i])
    ax.plot(x,range(y_size), lw=4)
    ax.imshow(specImg_abf_live.data)
    
fig.savefig('ADF_section.png')

In [ ]:
specImg_hl_bin = rebin_non_constant_interval(specImg_hl_temp1, ini_interval, axis)

In [ ]:
specImg_ll_bin = rebin_non_constant_interval(specImg_ll, ini_interval, axis)

In [ ]:
specImg_hl_bin.plot()

### Direct integration 3D datacube

### VCA

In [ ]:
specImg_hl.decomposition(algorithm='vca', p=3)

In [ ]:
cascade_plot = hs.plot.plot_spectra(specImg_hl.get_decomposition_factors(), style='cascade', linewidth=3.0, dpi=200, padding=0.15)
#cascade_plot.figure.savefig("VCA_3compo.png", dpi=200)

In [ ]:
for i in (0,1,2):
    specImg_hl.plot_bss_loadings(comp_ids=i+1, same_window=False, cmap='gnuplot')[0],
    plt.savefig('vca_no-deconv_comp' + str(i) + '.png')

### VCA after deconvolution

In [ ]:
specImg_hl_PCA8_deconv.decomposition(algorithm='vca', p=3)

In [ ]:
cascade_plot = hs.plot.plot_spectra(specImg_hl_PCA8_deconv.get_decomposition_factors(), style='cascade', dpi=200, padding=0.15)

In [ ]:
for i in (0,1,2):
    specImg_hl_PCA8_deconv.plot_bss_loadings(comp_ids=i+1, same_window=False, cmap='gnuplot')[0],
    plt.savefig('vca_RL8-deconv_comp' + str(i) + '.png')

#### Thickness map

In [ ]:
specImg_ll.estimate_elastic_scattering_threshold().plot()

In [ ]:
Thick_map = specImg_ll.estimate_thickness(threshold=specImg_ll.estimate_elastic_scattering_threshold())

In [ ]:
#Thick_map.plot()
plt.imsave('Thickness_map.png', Thick_map.data, cmap='gnuplot')

### Elemental maps

#### Si L edge

In [ ]:
Si_L_full = specImg_hl_PCA8.remove_background(signal_range=(75.,95.))

In [ ]:
Int_Si_L_full = Si_L_full.integrate_in_range((99.0,169.0))

In [ ]:
#Int_Si_L_full.plot()
plt.imsave('Si_L_edge.png', Int_Si_L_full.data, cmap='gnuplot')

#### C_K edge

In [ ]:
C_K_full = specImg_hl_PCA8.remove_background(signal_range=(250.,280.))

In [ ]:
C_K_full.plot()

In [ ]:
Int_C_K_full = C_K_full.integrate_in_range((280.,350.))

In [ ]:
#Int_C_K_full.plot()
plt.imsave('C_K_edge.png', Int_C_K_full.data, cmap='gnuplot')

### Extract of the Si L3, L2 and L1 edges, fit not perfect

In [ ]:
specImg_hl_bin_mean = specImg_hl_bin.mean(1)
specImg_ll_bin_mean = specImg_ll_bin.mean(1)

In [ ]:
specImg_hl_bin_mean.crop_signal1D((85.,250.))

In [ ]:
specImg_hl_bin_mean_bg = specImg_hl_bin_mean.remove_background(signal_range=(80.,95.))

In [ ]:
specImg_hl_bin_mean_bg.plot()

In [ ]:
threylhod_ll = specImg_ll_bin_mean.estimate_elastic_scattering_threshold()

In [ ]:
specImg_hl_bin_mean_dec = specImg_hl_bin_mean_bg.fourier_ratio_deconvolution(ll = specImg_ll_bin_mean, threshold=threylhod_ll)

In [ ]:
specImg_hl_bin_mean_dec.plot()

In [ ]:
specImg_hl_bin_mean_dec.add_elements(('Si',))

In [ ]:
m.clear()

In [ ]:
m = specImg_hl_bin_mean_dec.create_model(ll=specImg_ll_bin_mean, GOS='hydrogenic')

In [ ]:
m.components

In [ ]:
m.enable_fine_structure()

In [ ]:
m.disable_background()

In [ ]:
m.disable_free_onset_energy() #last changed

In [ ]:
m.disable_adjust_position()

In [ ]:
m.set_all_edges_intensities_positive()

In [ ]:
m.set_parameters_free()

In [ ]:
m.multifit(kind='smart')

In [ ]:
m.plot()

In [ ]:
m.disable_edges(('Si_L3','Si_L2','Si_L1'))
m.enable_edges(('Si_L3',))
m_Si_L3 = m.as_signal()

In [ ]:
m.disable_edges(('Si_L3','Si_L2','Si_L1'))
m.enable_edges(('Si_L2',))
m_Si_L2 = m.as_signal()

In [ ]:
m.disable_edges(('Si_L3','Si_L2','Si_L1'))
m.enable_edges(('Si_L1',))
m_Si_L1 = m.as_signal()

In [ ]:
m_Si_L2.save('Si_L2')

In [ ]:
m_Si_L3.save('Si_L3')

In [ ]:
m_Si_L1.save('Si_L1')